In [1]:
%config IPCompleter.greedy = True

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import random
import pprint

import nltk
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

from collections import defaultdict

### 1. Load NLTK and Test Dataset</font>
***

#### Loading Treebank tagged sentences using **universal** tagset

`VERB - verbs (all tenses and modes)
NOUN - nouns (common and proper)
PRON - pronouns 
ADJ - adjectives
ADV - adverbs
ADP - adpositions (prepositions and postpositions)
CONJ - conjunctions
DET - determiners
NUM - cardinal numbers
PRT - particles or other function words
X - other: foreign words, typos, abbreviations
. - punctuation
`

In [4]:
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [5]:
# observe a few tagged sentences from the corpora
print(nltk_data[:2])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


#### Loading Test Data

In [6]:
file_object = open(r"test-sentences.txt","r", encoding="latin1")
test_data = file_object.read()
test_data

"Android is a mobile operating system developed by Google.\nAndroid has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.\nGoogle and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.\nTwitter is an online news and social networking service on which users post and interact with messages known as tweets.\nBefore entering politics, Donald Trump was a domineering businessman and a television personality.\nThe 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.\nThis is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.\nShow me the cheapest round trips from Dallas to Atlanta\nI would like to see flights from Denver to Philadelphia.\nShow me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.\nNASA invited social media users to experience the launch of ICESAT-2 Satell

In [7]:
# number of words in the test dataset
test_data_words = nltk.word_tokenize(test_data)
len(test_data_words)

181

#### Tagging Test Dataset With NLTK POS Tagger

In [8]:
test_tagged_words = {}
test_tagged = nltk.pos_tag(test_data_words, tagset='universal')
universal_tagset = [
    'VERB', 'NOUN', 'PRON', 'ADJ', 'ADV', 
    'ADP', 'CONJ', 'DET', 'NUM', 'PRT', 'X', '.'
]

for utag in universal_tagset:
    test_tagged_words[utag] = sorted(
        set([word for (word, tag) in test_tagged if tag == utag]))

i = random.randrange(len(universal_tagset))

pprint.pprint('words with tagged with {}'.format(universal_tagset[i]))
pprint.pprint(test_tagged_words[universal_tagset[i]])

'words with tagged with ADJ'
['best-selling',
 'cheapest',
 'domineering',
 'first',
 'international',
 'mobile',
 'online',
 'social']


### 2. Split data into train and validation datasets
***

In [9]:
train_set, validation_set = train_test_split(nltk_data,
                                             test_size=0.05,
                                             random_state=1234)

print('Number of sentences in train dataset : {0}'.format(len(train_set)))
print('Number of sentences in validation dataset : {0}'.format(len(validation_set)))

Number of sentences in train dataset : 3718
Number of sentences in validation dataset : 196


In [10]:
train_tagged_words = [tup for sent in train_set for tup in sent]

In [11]:
tokens = [pair[0] for pair in train_tagged_words]
print('total number of words in the training set : {0}'.format(len(tokens)))

vocabulary = set(tokens)
print('total number of unique words in the training set: {0}'.format(len(vocabulary)))

total number of words in the training set : 95799
total number of unique words in the training set: 12073


In [12]:
all_tags = [pair[1] for pair in train_tagged_words]
unique_tags = sorted(set(all_tags))

print('number of tags in the universal tagset : {}'.format(len(unique_tags)))
print(unique_tags)

number of tags in the universal tagset : 12
['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


### 3. Helper Functions
***

#### Store number of times a tag 'T' appears in the training dataset

In [13]:
tag_count_dict = dict()

for utag in unique_tags:
    tag_list = [pair[1] for pair in train_tagged_words if pair[1] == utag]
    tag_count_dict[utag] = len(tag_list)
    
print(tag_count_dict)    

{'.': 11130, 'ADJ': 6063, 'ADP': 9387, 'ADV': 3052, 'CONJ': 2144, 'DET': 8269, 'NOUN': 27471, 'NUM': 3364, 'PRON': 2619, 'PRT': 3070, 'VERB': 12910, 'X': 6320}


#### List of Unknown Words in Validation Dataset

In [14]:
val_data_unknown_words = [word for sent in validation_set for (word, tag) in sent if word not in vocabulary]
print('number of unknown words in validation data set : {0}'.format(len(set(val_data_unknown_words))))

number of unknown words in validation data set : 335


#### List of Unknown Words in Test Dataset

In [15]:
test_data_unknown_words = [word for word in test_data_words if word not in vocabulary]
print('number of unknown words in test data set : {0}'.format(len(set(test_data_unknown_words))))

number of unknown words in test data set : 28


#### Calculate Number of Words correctly tagged in Test Dataset

In [17]:
def calc_test_dataset_accuracy(tagged_test_set):
    total_words = 0
    correct_tagged_words = 0

    for word, tag in tagged_test_set:
        try:
            list_for_tag = test_tagged_words[tag]
        except KeyError:
            list_for_tag = []

        total_words += 1

        if word in list_for_tag:
            correct_tagged_words += 1

    print('total words - {0}. correctly tagged words - {1}. accuracy - {2}'.
          format(total_words, correct_tagged_words,
                 correct_tagged_words / total_words))